# 專題（二）：訓練 Bert 新聞分類器並提升精準度

## 專案目標
- 目標：請試著建製 BertForSequenceClassification 看得懂的資料集 NewsDataset
- news_clustering_train.tsv 中有 1800 篇新聞，六種類別的新聞各 300 篇
- news_clustering_test.tsv 中有 600 篇新聞，六種類別的新聞各 100 篇
- 六種類別：體育、財經、科技、旅遊、農業、遊戲

## 實作提示
- STEP1 - STEP4：資料處理
- STEP5：創造 train_batch 函數
- STEP6：創造 evaluate 函數
- STEP7：組合以上元素開始訓練，如果正確 validation accuracy 應該可以超過 90% 以上

## 重要知識點：專題結束後你可以學會
- 了解 BERT 的 Sequence Classification 任務如何進行
- 使用 TRAIN / VALID DATA 來了解深度學習模型的訓練情形
- 了解預訓練模型在 NLP 上的威力

In [1]:
# 連接個人資料 讀取 ＰＴＴ 訓練資料和儲存模型
#先連接自己的GOOGLE DRIVE 為了要儲存資料和訓練模型
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days-part2/project_1_3/')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days-part2/project_1_3


In [3]:
!python --version

Python 3.7.11


In [4]:
!pip install torch
!pip install transformers
#!pip install -q transformers
# 設定 torchtext 版本 安裝完必須重新啟動執行階段
!pip install torchtext==0.6.0

In [5]:
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from transformers import BertTokenizer, BertForSequenceClassification

In [6]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [7]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

valid_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
valid_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [8]:
ALL_NEWS_CLASSES = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

In [9]:
MODEL_NAME = 'bert-base-chinese'

In [10]:
# 建立數據集
class NewsDataset(Dataset):
    def __init__(self, tokenizer, titles, classes):
        self.tokenizer = tokenizer
        self.indexes = []
        self.texts = []
        self.labels = []
        for index in titles:
            self.indexes.append(index)
            self.texts.append(titles[index])
            self.labels.append(classes[index])

    def __getitem__(self, idx):
        text = self.texts[idx]

        input = self.tokenizer(text, return_tensors='pt')
        label = torch.tensor(ALL_NEWS_CLASSES.index(self.labels[idx]))

        return input, label

    def __len__(self):
        return len(self.indexes)


def create_mini_batch(samples):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    labels = []
    for s in samples:
        input_ids.append(s[0]['input_ids'].squeeze(0))
        token_type_ids.append(s[0]['token_type_ids'].squeeze(0))
        attention_mask.append(s[0]['attention_mask'].squeeze(0))
        labels.append(s[1])

    # zero pad 到同一序列長度
    input_ids = pad_sequence(input_ids, batch_first=True)
    token_type_ids = pad_sequence(token_type_ids, batch_first=True)
    attention_mask = pad_sequence(attention_mask, batch_first=True)
 
    labels = torch.stack(labels)

    return input_ids, token_type_ids, attention_mask, labels

In [11]:
batch_size = 32

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

train_dataset = NewsDataset(tokenizer, train_titles, train_classes)
valid_dataset = NewsDataset(tokenizer, valid_titles, valid_classes)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    collate_fn=create_mini_batch,
    shuffle=True)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=batch_size,
    collate_fn=create_mini_batch)

In [12]:
def train_batch(model, data, optimizer, device):
    model.train()
    input_ids, token_type_ids, attention_mask, labels = [d.to(device) for d in data]

    # Code Here
    outputs = model(
        input_ids=input_ids,
        token_type_ids=token_type_ids,
        attention_mask=attention_mask,
        labels=labels
    )
    loss = outputs.loss
    # End

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

In [13]:
def evaluate(model, valid_loader):
    model.eval()
    device = 'cuda' if next(model.parameters()).is_cuda else 'cpu'

    tot_count = 0
    tot_loss = 0
    tot_correct = 0

    with torch.no_grad():
        for data in valid_loader:
            input_ids, token_type_ids, attention_mask, labels = [d.to(device) for d in data]

            # Code Here
            outputs = model(
                input_ids=input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            tot_count += input_ids.size(0)
            tot_loss += outputs.loss.item()
            tot_correct += (outputs.logits.argmax(dim=-1) == labels).sum().item()
            
            # End
    
    evaluation = {
        'loss': tot_loss / tot_count,
        'acc': tot_correct / tot_count
    }
    return evaluation

In [14]:
# 訓練模型
epochs = 5
lr = 0.0001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained( # Code Here
    MODEL_NAME, 
    num_labels=6, 
    return_dict=True
)

model.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')

    for i, train_data in enumerate(train_loader):
        loss = train_batch(model, train_data, optimizer, device)
        train_size = train_data[0].size(0)

        if i % 10 == 0:
            print('train_loss: ', loss / train_size)

    evaluation = evaluate(model, valid_loader)
    print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))

    scheduler.step()    

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

epoch: 1
train_loss:  0.060375235974788666
train_loss:  0.028474539518356323
train_loss:  0.014009310863912106
train_loss:  0.01560203731060028
train_loss:  0.018747208639979362
train_loss:  0.007590021472424269
valid_evaluation: loss=0.013293009257564941, acc=0.8916666666666667
epoch: 2
train_loss:  0.0150650255382061
train_loss:  0.007151269353926182
train_loss:  0.02121441625058651
train_loss:  0.01409535575658083
train_loss:  0.001541155157610774
train_loss:  0.014083714224398136
valid_evaluation: loss=0.014841606517632803, acc=0.8716666666666667
epoch: 3
train_loss:  0.0036476710811257362
train_loss:  0.005735544487833977
train_loss:  0.008524185977876186
train_loss:  0.002162025310099125
train_loss:  0.00396896293386817
train_loss:  0.0009688572026789188
valid_evaluation: loss=0.010937012293531249, acc=0.9016666666666666
epoch: 4
train_loss:  0.00845180731266737
train_loss:  0.0029194143135100603
train_loss:  0.0018559604650363326
train_loss:  0.005174604244530201
train_loss:  0.

In [15]:
evaluation

{'acc': 0.9016666666666666, 'loss': 0.011538949767903735}

train_loss:  0.002987815300002694

valid_evaluation: loss=0.010043316627852619, acc=0.9133333333333333